In [2]:
# ============================================================
# 1️⃣ Librerías
# ============================================================
import os
import glob
import geopandas as gpd
import pandas as pd
import duckdb
from shapely.geometry import Point
from google.colab import drive
import requests
from zipfile import ZipFile
from tqdm import tqdm
import math
import gc

# ============================================================
# 2️⃣ Conexión a Google Drive
# ============================================================
drive.mount('/content/gdrive')

# Rutas base
base_dir = "/content/gdrive/MyDrive/trabajoGrado"
entrada = os.path.join(base_dir, "ookla_colombia")
salida_filtrados = os.path.join(base_dir, "ookla_colombia_filtrado")
os.makedirs(salida_filtrados, exist_ok=True)

# ============================================================
# 3️⃣ Descargar límite de Colombia si no existe
# ============================================================
print("🗺️ Verificando límites geográficos de Colombia...")

zip_path = "/content/colombia_shapefile.zip"
folder_path = "/content/colombia_shapefile"
url = "https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_COL_shp.zip"

if not os.path.exists(folder_path):
    r = requests.get(url)
    with open(zip_path, "wb") as f:
        f.write(r.content)
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(folder_path)

colombia = gpd.read_file(os.path.join(folder_path, "gadm41_COL_0.shp")).to_crs("EPSG:4326")
limite_colombia = colombia.union_all()
print("✅ Límite cargado correctamente.\n")

# ============================================================
# 4️⃣ Función para decodificar quadkey → lat/lon
# ============================================================
def quadkey_to_latlon(quadkey):
    tile_x = tile_y = 0
    level = len(quadkey)
    for i in range(level):
        bit = level - i
        mask = 1 << (bit - 1)
        q = int(quadkey[i])
        if q & 1:
            tile_x |= mask
        if q & 2:
            tile_y |= mask
    n = 2.0 ** level
    lon_deg = tile_x / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * tile_y / n)))
    lat_deg = math.degrees(lat_rad)
    return lat_deg, lon_deg

# ============================================================
# 5️⃣ Validar existencia de Parquets en Drive
# ============================================================
print("🔍 Buscando archivos Parquet en entrada...")
archivos = glob.glob(os.path.join(entrada, "**", "*.parquet"), recursive=True)

if not archivos:
    raise FileNotFoundError(f"🚫 No se encontraron archivos .parquet en {entrada}")

print(f"📂 Se encontraron {len(archivos)} archivos en la carpeta de entrada.\n")

# ============================================================
# 6️⃣ Procesar Parquets si no hay filtrados
# ============================================================
archivos_filtrados = glob.glob(os.path.join(salida_filtrados, "*.parquet"))

if len(archivos_filtrados) == 0:
    print("⚙️ Iniciando filtrado de datos dentro del territorio colombiano...\n")
    batch_size = 3
    for i in range(0, len(archivos), batch_size):
        lote = archivos[i:i+batch_size]
        print(f"🔹 Lote {i//batch_size + 1}/{len(archivos)//batch_size + 1} ({len(lote)} archivos)")

        for archivo in tqdm(lote):
            nombre = os.path.basename(archivo)
            try:
                df = pd.read_parquet(archivo)

                if 'quadkey' in df.columns:
                    df['lat'], df['lon'] = zip(*df['quadkey'].apply(quadkey_to_latlon))
                    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")
                    gdf_col = gdf[gdf.geometry.within(limite_colombia)]

                    if not gdf_col.empty:
                        salida = os.path.join(salida_filtrados, nombre)
                        gdf_col.drop(columns='geometry').to_parquet(salida)
                else:
                    print(f"⚠️ {nombre} sin columna 'quadkey'.")
            except Exception as e:
                print(f"⚠️ Error procesando {nombre}: {e}")
        gc.collect()
else:
    print(f"✅ Se encontraron {len(archivos_filtrados)} archivos ya filtrados. No se reprocesan.\n")



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
🗺️ Verificando límites geográficos de Colombia...
✅ Límite cargado correctamente.

🔍 Buscando archivos Parquet en entrada...
📂 Se encontraron 24 archivos en la carpeta de entrada.

✅ Se encontraron 24 archivos ya filtrados. No se reprocesan.



In [12]:
import duckdb
import os
from glob import glob
from tqdm import tqdm

# 📂 Ruta de tu carpeta de Google Drive
carpeta_base = "/content/gdrive/MyDrive/trabajoGrado"
carpeta_filtrados = os.path.join(carpeta_base, "ookla_colombia_filtrado")

# ✅ Conexión persistente a DuckDB
db_path = os.path.join(carpeta_base, "ookla_colombia.duckdb")
print(f"📂 Conectando a base persistente en: {db_path}")
conn = duckdb.connect(database=db_path, read_only=False)

# 🔍 Buscar archivos parquet filtrados
archivos_filtrados = sorted(glob(os.path.join(carpeta_filtrados, "*.parquet")))
if not archivos_filtrados:
    raise FileNotFoundError(f"⚠️ No se encontraron archivos Parquet en: {carpeta_filtrados}")
else:
    print(f"✅ Se encontraron {len(archivos_filtrados)} archivos Parquet para consolidar.")

# 🧱 Crear o reemplazar tabla combinando todos los archivos
print("🧱 Creando tabla 'ookla_filtrada' en DuckDB con unión por nombre...")

conn.execute(f"""
    CREATE OR REPLACE TABLE ookla_filtrada AS
    SELECT * FROM read_parquet('{carpeta_filtrados}/*.parquet', union_by_name=True);
""")

# 🔍 Validar carga
conteo = conn.execute("SELECT COUNT(*) FROM ookla_filtrada").fetchone()[0]
columnas = conn.execute("PRAGMA table_info('ookla_filtrada')").fetchdf()

print(f"📊 Total de registros cargados: {conteo:,}")
print(f"📑 Total de columnas consolidadas: {len(columnas)}")
print(f"🧾 Ejemplo de columnas: {list(columnas['name'][:10])}")

# 💾 Guardar cambios y cerrar conexión
conn.close()
print("\n✅ Base DuckDB consolidada correctamente.")


📂 Conectando a base persistente en: /content/gdrive/MyDrive/trabajoGrado/ookla_colombia.duckdb
✅ Se encontraron 24 archivos Parquet para consolidar.
🧱 Creando tabla 'ookla_filtrada' en DuckDB con unión por nombre...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

📊 Total de registros cargados: 594,175
📑 Total de columnas consolidadas: 14
🧾 Ejemplo de columnas: ['quadkey', 'tile', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'lat', 'lon', '__index_level_0__']

✅ Base DuckDB consolidada correctamente.


In [13]:
# ============================================================
# 8️⃣ Validar si faltan archivos por procesar
# ============================================================
faltantes = [f for f in archivos if os.path.basename(f) not in [os.path.basename(x) for x in archivos_filtrados]]
if len(faltantes) > 0:
    print(f"⚠️ Faltan {len(faltantes)} archivos por filtrar. Ejemplo: {faltantes[:3]}")
else:
    print("🎯 Todos los archivos fueron procesados y almacenados en DuckDB.")

🎯 Todos los archivos fueron procesados y almacenados en DuckDB.


In [22]:
import duckdb
conn = duckdb.connect("/content/gdrive/MyDrive/trabajoGrado/ookla_colombia.duckdb")

# Ver primeras filas
df = conn.execute("SELECT * FROM ookla_filtrada").fetchdf()
df.head()


,quadkey,tile,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,lat,lon,__index_level_0__,tile_x,tile_y,avg_lat_down_ms,avg_lat_up_ms
0,0322032133322112,"POLYGON((-81.705322265625 12.5920935246744, -8...",1480,2597,325,125,18,12.592094,-81.705322,2004888,NaN,NaN,NaN,NaN
1,0322032133322121,"POLYGON((-81.7108154296875 12.5867324324641, -...",1427,4024,359,123,12,12.586732,-81.710815,2004891,NaN,NaN,NaN,NaN
2,0322032133322122,"POLYGON((-81.71630859375 12.5813712282457, -81...",950,1102,172,14,8,12.581371,-81.716309,2004892,NaN,NaN,NaN,NaN
3,0322032133322123,"POLYGON((-81.7108154296875 12.5813712282457, -...",900,884,262,7,3,12.581371,-81.710815,2004893,NaN,NaN,NaN,NaN
4,0322032133322130,"POLYGON((-81.705322265625 12.5867324324641, -8...",1180,2091,127,29,17,12.586732,-81.705322,2004894,NaN,NaN,NaN,NaN


In [23]:
df.shape

(594175, 14)

validar y descargar shapes

In [24]:
import os
import requests

# 📂 Ruta donde deben estar los archivos en Google Drive
ruta_shapes = "/content/gdrive/MyDrive/trabajoGrado/shapes"
os.makedirs(ruta_shapes, exist_ok=True)

# 🗺️ Archivos a validar
archivos_shapes = {
    "colombia_departamentos.geojson": "https://raw.githubusercontent.com/marcovega/colombia-json/master/colombia.geo.json",
    "colombia_municipios.geojson": "https://github.com/marcovega/colombia-json/raw/master/colombia_municipios.geo.json"
}

# ✅ Validar existencia o descargar si no existen
for nombre, url in archivos_shapes.items():
    ruta_archivo = os.path.join(ruta_shapes, nombre)

    if os.path.exists(ruta_archivo):
        print(f"✅ El archivo '{nombre}' ya existe en la ruta: {ruta_shapes}")
    else:
        print(f"⬇️ Descargando '{nombre}' desde {url} ...")
        try:
            response = requests.get(url)
            response.raise_for_status()
            with open(ruta_archivo, "wb") as f:
                f.write(response.content)
            print(f"✅ Archivo '{nombre}' descargado correctamente.")
        except Exception as e:
            print(f"❌ Error al descargar '{nombre}': {e}")

print("\n🏁 Validación de shapes completada.")


✅ El archivo 'colombia_departamentos.geojson' ya existe en la ruta: /content/gdrive/MyDrive/trabajoGrado/shapes
✅ El archivo 'colombia_municipios.geojson' ya existe en la ruta: /content/gdrive/MyDrive/trabajoGrado/shapes

🏁 Validación de shapes completada.


Registro de shapes en DuckDB

In [29]:
import os
import geopandas as gpd
import duckdb

# ==========================
# 1️⃣ Definir rutas
# ==========================
shapes_dir = "/content/gdrive/MyDrive/trabajoGrado/shapes"
deptos_path = os.path.join(shapes_dir, "colombia_departamentos.geojson")
mpios_path = os.path.join(shapes_dir, "colombia_municipios.geojson")

# ==========================
# 2️⃣ Validar existencia de archivos
# ==========================
for path in [deptos_path, mpios_path]:
    if os.path.exists(path):
        print(f"✅ Archivo encontrado: {os.path.basename(path)}")
    else:
        print(f"❌ No se encontró {os.path.basename(path)} en {shapes_dir}. "
              f"Por favor descarga el archivo y guárdalo allí.")

# ==========================
# 3️⃣ Cargar los GeoJSON en GeoPandas
# ==========================
print("\n📥 Cargando shapes en GeoPandas...")
gdf_deptos = gpd.read_file(deptos_path)
gdf_mpios = gpd.read_file(mpios_path)

# 💡 Convertir geometría a texto (WKT) antes de registrar
gdf_deptos["geometry_wkt"] = gdf_deptos.geometry.to_wkt()
gdf_mpios["geometry_wkt"] = gdf_mpios.geometry.to_wkt()

# Eliminar la columna geometry original
gdf_deptos = gdf_deptos.drop(columns="geometry")
gdf_mpios = gdf_mpios.drop(columns="geometry")

# ==========================
# 4️⃣ Crear base DuckDB y registrar tablas
# ==========================
print("\n🧱 Creando tablas 'departamentos' y 'municipios' en DuckDB...")
conn = duckdb.connect("/content/gdrive/MyDrive/trabajoGrado/ookla_colombia.duckdb", read_only=False)

conn.register("gdf_deptos", gdf_deptos)
conn.register("gdf_mpios", gdf_mpios)

# Crear tablas en DuckDB usando geometría en WKT
conn.execute("""
CREATE OR REPLACE TABLE departamentos AS
SELECT *, ST_GeomFromText(geometry_wkt) AS geom FROM gdf_deptos
""")

conn.execute("""
CREATE OR REPLACE TABLE municipios AS
SELECT *, ST_GeomFromText(geometry_wkt) AS geom FROM gdf_mpios
""")

# ==========================
# 5️⃣ Listar tablas disponibles
# ==========================
print("\n📋 Tablas registradas en DuckDB:")
print(conn.execute("SHOW TABLES").fetchdf())

# ==========================
# 6️⃣ Detectar el nombre correcto del municipio
# ==========================
cols_mpios = [c.lower() for c in conn.execute("PRAGMA table_info('municipios')").fetchdf()["name"].tolist()]
possible_name_cols = ["nombre", "nom_mpio", "mpio_cnmbre", "municipio", "mpio_nombre"]
col_mpio = next((c for c in cols_mpios if c in possible_name_cols), cols_mpios[0])

# ==========================
# 7️⃣ Vista previa adaptativa
# ==========================
print(f"\n🗺️ Vista previa de 'municipios' (columna de nombre detectada: '{col_mpio}'):")
query_preview = f"""
SELECT
    "{col_mpio}" AS municipio_nombre,
    LEFT(ST_AsText(geom), 80) AS geom_preview
FROM municipios
LIMIT 3
"""
print(conn.execute(query_preview).fetchdf())

print("\n✅ Shapes registrados y verificados exitosamente en la base 'ookla_colombia.duckdb'")


✅ Archivo encontrado: colombia_departamentos.geojson
✅ Archivo encontrado: colombia_municipios.geojson

📥 Cargando shapes en GeoPandas...

🧱 Creando tablas 'departamentos' y 'municipios' en DuckDB...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


📋 Tablas registradas en DuckDB:
             name
0   departamentos
1      gdf_deptos
2       gdf_mpios
3      municipios
4  ookla_filtrada

🗺️ Vista previa de 'municipios' (columna de nombre detectada: 'shapename'):
  municipio_nombre                                       geom_preview
0       San Rafael  POLYGON ((-74.828222 6.338663, -74.828545 6.33...
1        San Roque  POLYGON ((-74.828222 6.338663, -74.82777 6.338...
2      San Vicente  POLYGON ((-75.306647 6.392053, -75.306669 6.39...

✅ Shapes registrados y verificados exitosamente en la base 'ookla_colombia.duckdb'


In [30]:
print("📊 Conteo de registros en cada shape:")
print(conn.execute("SELECT COUNT(*) AS total_deptos FROM departamentos").fetchdf())
print(conn.execute("SELECT COUNT(*) AS total_mpios FROM municipios").fetchdf())


📊 Conteo de registros en cada shape:
   total_deptos
0            33
   total_mpios
0         1122


In [31]:
print("\n🧩 Columnas en 'departamentos':")
print(conn.execute("PRAGMA table_info('departamentos')").fetchdf())

print("\n🧩 Columnas en 'municipios':")
print(conn.execute("PRAGMA table_info('municipios')").fetchdf())



🧩 Columnas en 'departamentos':
   cid          name      type  notnull dflt_value     pk
0    0     shapeName   VARCHAR    False       None  False
1    1      shapeISO   VARCHAR    False       None  False
2    2       shapeID   VARCHAR    False       None  False
3    3    shapeGroup   VARCHAR    False       None  False
4    4     shapeType   VARCHAR    False       None  False
5    5  geometry_wkt   VARCHAR    False       None  False
6    6          geom  GEOMETRY    False       None  False

🧩 Columnas en 'municipios':
   cid          name      type  notnull dflt_value     pk
0    0     shapeName   VARCHAR    False       None  False
1    1      shapeISO   VARCHAR    False       None  False
2    2       shapeID   VARCHAR    False       None  False
3    3    shapeGroup   VARCHAR    False       None  False
4    4     shapeType   VARCHAR    False       None  False
5    5  geometry_wkt   VARCHAR    False       None  False
6    6          geom  GEOMETRY    False       None  False


In [34]:
import duckdb

# Cierra la conexión anterior si existe
try:
    conn.close()
    print("🔒 Conexión anterior cerrada correctamente.")
except:
    print("⚠️ No había conexión activa.")

# Conectarse en modo lectura
conn = duckdb.connect("/content/gdrive/MyDrive/trabajoGrado/ookla_colombia.duckdb", read_only=True)

# Listar todas las tablas persistentes
print("📋 Tablas existentes en la base DuckDB:")
print(conn.execute("SHOW TABLES").fetchdf())


🔒 Conexión anterior cerrada correctamente.
📋 Tablas existentes en la base DuckDB:
             name
0   departamentos
1      municipios
2  ookla_filtrada


unión espacial y creación de nueva tabla enriquecida

In [36]:
import duckdb
import os

# Ruta de tu base y shapes
db_path = "/content/gdrive/MyDrive/trabajoGrado/ookla_colombia.duckdb"
shapes_path = "/content/gdrive/MyDrive/trabajoGrado/shapes"

# Cerrar cualquier conexión anterior
try:
    conn.close()
    print("🔒 Conexión anterior cerrada correctamente.")
except:
    pass

# Conectar a la base
conn = duckdb.connect(db_path)
print("✅ Conectado a la base en modo escritura.")

# Cargar extensión espacial (necesaria para ST_Within)
conn.execute("INSTALL spatial;")
conn.execute("LOAD spatial;")
print("🌍 Extensión 'spatial' cargada correctamente.")

# Revisar las tablas existentes
tablas = conn.execute("SHOW TABLES;").fetchdf()
print("\n📋 Tablas existentes en la base:")
print(tablas)

# Confirmar que ookla_filtrada existe
if "ookla_filtrada" not in tablas["name"].values:
    raise ValueError("⚠️ La tabla 'ookla_filtrada' no existe en la base.")

# Crear geometría a partir de lat/lon
conn.execute("""
    CREATE OR REPLACE TABLE ookla_filtrada_geo AS
    SELECT *,
           ST_Point(lon, lat) AS geom
    FROM ookla_filtrada;
""")
print("📍 Geometría creada en 'ookla_filtrada_geo'.")

# Crear tabla enriquecida con municipio y departamento
print("\n🧭 Creando tabla enriquecida con municipios y departamentos...")
conn.execute("""
    CREATE OR REPLACE TABLE ookla_geo AS
    SELECT
        o.*,
        m.shapeName AS municipio,
        d.shapeName AS departamento
    FROM ookla_filtrada_geo o
    LEFT JOIN municipios m
        ON ST_Within(o.geom, m.geom)
    LEFT JOIN departamentos d
        ON ST_Within(o.geom, d.geom);
""")

print("✅ Tabla 'ookla_geo' creada exitosamente con datos geográficos.")

# Contar registros de validación
conteo = conn.execute("SELECT COUNT(*) AS total FROM ookla_geo;").fetchdf()
print(f"\n📊 Total de registros en 'ookla_geo': {conteo.iloc[0,0]}")

# Cerrar conexión
conn.close()
print("\n🔒 Conexión cerrada correctamente.")


🔒 Conexión anterior cerrada correctamente.
✅ Conectado a la base en modo escritura.
🌍 Extensión 'spatial' cargada correctamente.

📋 Tablas existentes en la base:
             name
0   departamentos
1      municipios
2  ookla_filtrada


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

📍 Geometría creada en 'ookla_filtrada_geo'.

🧭 Creando tabla enriquecida con municipios y departamentos...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Tabla 'ookla_geo' creada exitosamente con datos geográficos.

📊 Total de registros en 'ookla_geo': 594175

🔒 Conexión cerrada correctamente.


In [37]:
import duckdb
conn = duckdb.connect("/content/gdrive/MyDrive/trabajoGrado/ookla_colombia.duckdb")

# Ver primeras filas
df = conn.execute("SELECT * FROM ookla_geo").fetchdf()
df.head()


,quadkey,tile,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,lat,lon,__index_level_0__,tile_x,tile_y,avg_lat_down_ms,avg_lat_up_ms,geom,municipio,departamento
0,0322032133322112,"POLYGON((-81.705322265625 12.5920935246744, -8...",1480,2597,325,125,18,12.592094,-81.705322,2004888,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",San Andrés,"Archipiélago de San Andrés, Providencia y Sant..."
1,0322032133322121,"POLYGON((-81.7108154296875 12.5867324324641, -...",1427,4024,359,123,12,12.586732,-81.710815,2004891,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",San Andrés,"Archipiélago de San Andrés, Providencia y Sant..."
2,0322032133322122,"POLYGON((-81.71630859375 12.5813712282457, -81...",950,1102,172,14,8,12.581371,-81.716309,2004892,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",San Andrés,"Archipiélago de San Andrés, Providencia y Sant..."
3,0322032133322123,"POLYGON((-81.7108154296875 12.5813712282457, -...",900,884,262,7,3,12.581371,-81.710815,2004893,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",San Andrés,"Archipiélago de San Andrés, Providencia y Sant..."
4,0322032133322130,"POLYGON((-81.705322265625 12.5867324324641, -8...",1180,2091,127,29,17,12.586732,-81.705322,2004894,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",San Andrés,"Archipiélago de San Andrés, Providencia y Sant..."


In [38]:
df.shape

(594175, 17)

Conteo total de registros

In [39]:
conn.execute("SELECT COUNT(*) AS total_registros FROM ookla_geo;").fetchdf()


,total_registros
0,594175


Cantidad de registros por municipio

In [40]:
conn.execute("""
SELECT
    municipio,
    COUNT(*) AS total_registros
FROM ookla_geo
GROUP BY municipio
ORDER BY total_registros DESC;
""").fetchdf()


,municipio,total_registros
0,"Bogotá, D.c.",27526
1,Cali,13841
2,Medellín,12135
3,Villavicencio,9347
4,Pereira,9059
...,...,...
1040,Mapiripana,2
1041,San Felipe,2
1042,Puerto Alegría,2
1043,Cacahual,1


Cantidad de registros por departamento

In [41]:
conn.execute("""
SELECT
    departamento,
    COUNT(*) AS total_registros
FROM ookla_geo
GROUP BY departamento
ORDER BY total_registros DESC;
""").fetchdf()


,departamento,total_registros
0,Antioquia,91561
1,Cundinamarca,76079
2,Valle del Cauca,56385
3,Santander,37451
4,Huila,30310
5,Bogota Capital District,27447
6,Boyacá,26485
7,Cauca,24676
8,Tolima,23505
9,Meta,23342


municipios dentro de cada departamento

In [42]:
conn.execute("""
SELECT
    departamento,
    municipio,
    COUNT(*) AS total_registros
FROM ookla_geo
GROUP BY departamento, municipio
ORDER BY departamento, total_registros DESC;
""").fetchdf()


,departamento,municipio,total_registros
0,Amazonas,Leticia,376
1,Amazonas,Puerto Nariño,22
2,Amazonas,Santander (Araracuara),11
3,Amazonas,El Encanto,8
4,Amazonas,La Chorrera,7
...,...,...,...
1248,None,Moñitos,2
1249,None,San Miguel (La Dorada),1
1250,None,Tumaco,1
1251,None,Juradó,1


Municipios o departamentos sin registros (si los hay)

In [43]:
# Municipios sin registros
conn.execute("""
SELECT shapeName AS municipio
FROM municipios
WHERE shapeName NOT IN (SELECT DISTINCT municipio FROM ookla_geo);
""").fetchdf()

# Departamentos sin registros
conn.execute("""
SELECT shapeName AS departamento
FROM departamentos
WHERE shapeName NOT IN (SELECT DISTINCT departamento FROM ookla_geo);
""").fetchdf()


,departamento


promedio de velocidad por departamento

In [44]:
conn.execute("""
SELECT
    departamento,
    ROUND(AVG(avg_d_kbps), 2) AS promedio_bajada_kbps,
    ROUND(AVG(avg_u_kbps), 2) AS promedio_subida_kbps,
    COUNT(*) AS total_registros
FROM ookla_geo
GROUP BY departamento
ORDER BY promedio_bajada_kbps DESC;
""").fetchdf()


,departamento,promedio_bajada_kbps,promedio_subida_kbps,total_registros
0,Bogota Capital District,99724.44,65107.59,27447
1,Atlántico,72341.94,50324.47,13543
2,"Archipiélago de San Andrés, Providencia y Sant...",63303.26,57568.89,1098
3,Antioquia,57106.13,34092.90,91561
4,Risaralda,49297.12,25261.55,15691
5,Guainía,49262.51,13324.11,356
6,Quindío,47358.11,26616.92,10508
7,Cesar,47319.17,32073.35,9998
8,Vaupés,46464.44,15427.98,180
9,Norte de Santander,46378.89,29749.19,14761
